# nb-model_exp-results-final

In [1]:
import sys
import os
from glob import glob, iglob
from os import sep
from os.path import dirname, realpath, exists, basename
from pathlib import Path
from distutils.dir_util import copy_tree
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_exp-results-final.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
import optuna

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 50)

from common_util import MODEL_DIR, dump_df, benchmark, NestedDefaultDict, load_json, makedir_if_not_exists
from model.common import ASSETS, INTERVAL_YEARS, INTRADAY_LEN, EXP_LOG_DIR, EXP_PARAMS_DIR
from model.common import OPTUNA_DB_FNAME, OPTUNA_N_TRIALS, OPTUNA_TIMEOUT_HOURS
from recon.viz import *
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_exp-results-final.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Final Results

In [2]:
def get_bench(pattern):
    ndd = NestedDefaultDict()
    for path in iglob(pattern, recursive=True):
        ndd[path.split('/')] = load_json(path)

    df_idx = [k[-3:-1] for k in ndd.keys()]
    midx = pd.MultiIndex.from_tuples([k[-3:-1] for k in ndd.keys()], names=['asset', 'data'])
    df = pd.DataFrame.from_records([b['benchmark-hold'] for b in ndd.values()], index=midx)

    return df

In [3]:
def get_final(split, data_names):
    res = []
    for asset_name in ASSETS:
        asset_dir = f"{test_dir}{asset_name}{sep}"

        for data_name in data_names:
            paths = [path for path in iglob(f"{asset_dir}/**/{data_name}/**/{split}.json", recursive=True)]
            idxs = list(map(lambda p: p.replace(asset_dir, '').replace(data_name+sep, '').replace(f"{sep}{split}.json", ''), paths))
            idx = [(asset_name, data_name, *i.split(sep)) for i in idxs]
            midx = pd.MultiIndex.from_tuples(idx, names=['asset', 'data', 'model', 'study', 'trial'])
            r = list(map(load_json, paths))
            res.append(pd.DataFrame.from_records(r, index=midx))
    return pd.concat(res)

In [4]:
def get_sel(split):
    return [f'{split}_clf_accuracy',
           f'{split}_kelly_sharpe', f'{split}_kelly_cagr',
           f'{split}_binary_sharpe', f'{split}_binary_cagr',
           f'{split}_conf_sharpe', f'{split}_conf_cagr',
           f'{split}_binary_longfreq']

In [5]:
def get_best(res_df, key, sel):
    max_idx = res_df.groupby('model')[key].idxmax()
    return res_df.loc[max_idx, sel]

In [6]:
def bench_diff(res_df, bench_df, split):
    res_df[f'{split}_clf_accuracy'] -= bench_df[f'{split}_clf_accuracy']
    for retmet in ['sharpe', 'cagr']:
        res_df[f'{split}_kelly_{retmet}'] -= bench_df[f'{split}_{retmet}']
        res_df[f'{split}_binary_{retmet}'] -= bench_df[f'{split}_{retmet}']
        res_df[f'{split}_conf_{retmet}'] -= bench_df[f'{split}_{retmet}']
    return res_df

In [7]:
def add_style(df):
    df = df.style \
         .background_gradient(subset=[c for c in df.columns if c.endswith(('accuracy', 'f1', 'profit', 'sharpe', 'cagr'))], cmap='BuGn')
#     .set_sticky(axis="index") \
    return df

In [8]:
sm_name = 'anp'
model_name = 'np'
# data_names = ['2009_2018_ddir_h_pba_h', '2009_2018_ddir_h_pba_h_h_vol_h']
data_names = ['2007_2018_ddir_h_pba_h', '2007_2018_ddir_h_pba_h_h_vol_h']
#test_dir = EXP_LOG_DIR +sep.join(['final_3', sm_name]) +sep
test_dir = EXP_LOG_DIR +sep.join(['resfinal_5', sm_name]) +sep

bench = {split: get_bench(f"{EXP_LOG_DIR}/bench/**/{split}.json") for split in ['train', 'val', 'test']}
final = {split: get_final(split, data_names) for split in ['train', 'val', 'test']}

In [9]:
datas = data_names[:2]
year = datas[0][:4]
split = 'test'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
params = load_json("params.json", EXP_LOG_DIR+'resfinal_5')

res = []
for asset_name in ASSETS:
    b = benchyear.loc[asset_name]
    asset_res = []
    for data_name in datas:
        for model_name in ["base", "cnp", "lnp", "np"]:
            p = params[asset_name][data_name][model_name]
            best = final[split].xs((asset_name, data_name, model_name), drop_level=False)
            if (len(lvl := best.index.get_level_values(-1)) > 0):
                pv = [v for v in lvl if (p and v.endswith(p))]
                if (len(pv) > 0):
                    best = best.xs(pv[0], level=-1, drop_level=False).loc[:, sel]
                    asset_res.append(best)
                else:
                    pass
                    #best = best.xs(lvl[0], level=-1, drop_level=False).replace(value=0.0, regex=r'^.*$').loc[:, sel] 
                    #asset_res.append(best)
                
    asset_res = pd.concat(asset_res)
    res.append(asset_res)
    #display(add_style(asset_res))
    #best = bench_diff(best, b, split)
res = pd.concat(res)
display(add_style(res))

/home/kev/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2947: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [10]:
datas = data_names[:2]
year = datas[0][:4]
split = 'val'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
params = load_json("params.json", EXP_LOG_DIR+'resfinal_5')

res = []
for asset_name in ASSETS:
    b = benchyear.loc[asset_name]
    asset_res = []
    for data_name in datas:
        for model_name in ["base", "cnp", "lnp", "np"]:
            p = params[asset_name][data_name][model_name]
            best = final[split].xs((asset_name, data_name, model_name), drop_level=False)
            if (len(lvl := best.index.get_level_values(-1)) > 0):
                pv = [v for v in lvl if (p and v.endswith(p))]
                if (len(pv) > 0):
                    best = best.xs(pv[0], level=-1, drop_level=False).loc[:, sel]
                    asset_res.append(best)
    asset_res = pd.concat(asset_res)
    res.append(asset_res)
    #display(add_style(asset_res))
    #best = bench_diff(best, b, split)
res = pd.concat(res)
display(add_style(res))

In [11]:
datas = data_names[:2]
year = datas[0][:4]
split = 'train'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
display(year)
params = load_json("params.json", EXP_LOG_DIR+'resfinal_5')

res = []
for asset_name in ASSETS:
    display(asset_name)
    b = benchyear.loc[asset_name]
    asset_res = []
    for data_name in datas:
        for model_name in ["base", "cnp", "lnp", "np"]:
            p = params[asset_name][data_name][model_name]
            best = final[split].xs((asset_name, data_name, model_name), drop_level=False)
            if (len(lvl := best.index.get_level_values(-1)) > 0):
                pv = [v for v in lvl if (p and v.endswith(p))]
                if (len(pv) > 0):
                    best = best.xs(pv[0], level=-1, drop_level=False).loc[:, sel]
                    asset_res.append(best)
    asset_res = pd.concat(asset_res)
    res.append(asset_res)
    #display(add_style(asset_res))
    #best = bench_diff(best, b, split)
res = pd.concat(res)
display(add_style(res))

'2007'

'sp_500'

'russell_2000'

'nasdaq_100'

'dow_jones'

In [12]:
datas = data_names[:2]
year = datas[0][:4]
split = 'test'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
display(year)

for asset_name in ASSETS:
    display(asset_name)
    b = benchyear.loc[asset_name]
    for data_name in datas:
        best = get_best(final[split].xs((asset_name, data_name)), key, sel)
        best = bench_diff(best, b, split)
        display(add_style(best))

'2007'

'sp_500'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,471,7_000414,-0.006250,0.574439,-0.006422,0.399948,0.047104,0.459879,0.020853,0.614062
cnp,476,8_000655,-0.009375,-0.563546,-0.044003,-0.285453,-0.032772,-0.299779,-0.036982,0.946875
lnp,476,5_000111,-0.012500,0.458327,-0.038517,0.286864,0.033797,0.299328,-0.001494,0.315625
np,476,4_000507,-0.006250,0.344562,-0.040595,0.785583,0.095081,0.780038,0.026673,0.493750


,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,376,9_000190,0.014063,0.967469,-0.006164,1.481762,0.185626,1.422015,0.086796,0.501563
cnp,482,7_000769,0.023438,0.535037,-0.039824,0.747123,0.089656,0.744125,0.024509,0.901563
lnp,376,1_000941,0.000000,0.486154,-0.037138,0.470322,0.055968,0.475878,0.009856,0.631250
np,376,1_000941,0.001563,0.372827,-0.036173,0.081606,0.009445,0.105176,-0.012231,0.989062


'russell_2000'

...

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,455,7_000202,-0.057813,0.468121,-0.027874,0.564195,0.076329,0.552255,0.023969,0.604688
cnp,452,1_000177,-0.056250,0.461966,-0.045577,0.403631,0.054067,0.436050,0.004482,0.646875
lnp,452,2_000087,-0.060938,0.442900,-0.049578,0.593582,0.080461,0.604767,0.015010,0.445312
np,455,6_000127,-0.064063,0.120222,-0.048501,0.032631,0.004116,0.065227,-0.021347,0.520312


,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,420,7_000963,-0.019819,0.390362,-0.028991,0.102187,0.014158,0.187297,-0.006665,0.633224
cnp,425,2_000000,-0.041201,0.524099,-0.048354,0.267826,0.036659,0.322830,-0.005201,0.536184
lnp,420,9_000808,-0.026398,1.155357,0.002272,0.836107,0.116920,0.944127,0.059248,0.643092
np,423,5_000906,-0.064227,0.495212,-0.030250,0.036530,0.005417,0.157227,-0.011479,0.523026


'dow_jones'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,342,5_000515,-0.050000,0.261887,-0.053124,-0.230282,-0.027776,-0.152624,-0.039304,0.437500
cnp,342,0_000644,-0.001562,-0.498248,-0.068676,-0.084400,-0.010559,-0.100736,-0.038540,0.320312
lnp,342,5_000515,-0.025000,0.468608,-0.045994,-0.194472,-0.023573,-0.077419,-0.033667,0.568750
np,342,1_000675,-0.028125,0.074449,-0.066425,-0.697997,-0.081227,-0.668784,-0.072375,0.693750


,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,445,3_000944,-0.005099,-0.074663,-0.057496,0.124772,0.016092,0.101248,-0.020160,0.702303
cnp,447,0_000457,-0.049507,-0.618103,-0.069135,-0.306895,-0.036216,-0.355909,-0.051683,0.555921
lnp,445,8_000737,-0.016612,0.039571,-0.043635,0.309316,0.039219,0.256711,-0.002139,0.585526
np,447,2_000010,-0.001809,-0.067603,-0.042682,0.025606,0.003867,0.001117,-0.019047,0.751645


In [11]:
sm_name = 'anp'
model_name = 'np'
data_names = ['1996_2018_ddir_h_pba_h']
#test_dir = EXP_LOG_DIR +sep.join(['final_3', sm_name]) +sep
test_dir = EXP_LOG_DIR +sep.join(['final', sm_name]) +sep

bench = {split: get_bench(f"{EXP_LOG_DIR}/bench/**/{split}.json") for split in ['train', 'val', 'test']}
final = {split: get_final(split, data_names) for split in ['train', 'val', 'test']}

In [12]:
datas = data_names[:2]
year = datas[0][:4]
split = 'test'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
display(year)

for asset_name in ASSETS:
    display(asset_name)
    b = benchyear.loc[asset_name]
    for data_name in datas:
        best = get_best(final[split].xs((asset_name, data_name)), key, sel)
        best = bench_diff(best, b, split)
        display(add_style(best))

'1996'

'sp_500'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,471,1_000110,-0.031250,-0.229629,-0.074218,-0.359223,-0.042299,-0.332089,-0.057230,0.488567
cnp,476,5_000111,-0.023628,-0.222270,-0.078557,0.127500,0.015226,0.103712,-0.031371,0.410061
lnp,471,0_000474,-0.012195,-0.096233,-0.067988,0.232577,0.027812,0.198589,-0.020058,0.513720
np,471,3_000608,0.000000,0.042844,-0.062361,-0.020728,-0.002718,-0.001753,-0.032004,1.000000


'russell_2000'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,440,4_001000,-0.015244,0.080177,-0.059447,0.187021,0.029069,0.184899,-0.014299,0.510671
cnp,445,2_000891,0.000762,0.167080,-0.078429,0.045371,0.007006,0.048459,-0.034047,0.999238
lnp,442,4_000141,-0.016768,0.177243,-0.043422,0.066978,0.010355,0.104452,-0.015556,0.542683
np,442,6_000206,0.001524,0.334315,-0.077184,0.567704,0.090366,0.566580,0.005978,0.803354


'nasdaq_100'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,452,5_000035,-0.075457,0.251312,-0.059927,-0.110365,-0.013828,-0.079043,-0.035515,0.283537
cnp,452,0_000940,-0.031250,0.636432,-0.038644,0.722300,0.094927,0.736676,0.027464,0.515244
lnp,452,1_000177,-0.043445,0.544501,-0.045251,0.644556,0.084362,0.653385,0.019053,0.445122
np,452,2_000087,-0.044969,0.321201,-0.047915,-0.064510,-0.008099,0.005093,-0.026854,0.663110


'dow_jones'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,342,3_000923,-0.012957,-0.272332,-0.092107,0.044254,0.005337,0.034774,-0.043004,0.685976
cnp,342,1_000675,0.003049,0.131671,-0.084360,0.054561,0.006601,0.069048,-0.038483,0.996189
lnp,342,3_000923,-0.034299,-0.325460,-0.093200,-0.435487,-0.052121,-0.428949,-0.071362,0.440549
np,342,1_000675,0.004573,0.064093,-0.084148,0.078043,0.009490,0.084999,-0.037011,0.983994


In [15]:
sm_name = 'anp'
model_name = 'np'
data_names = ['2007_2018_ddir_h_pba_h', '2007_2018_ddir_h_pba_h_h_vol_h']
#test_dir = EXP_LOG_DIR +sep.join(['final_3', sm_name]) +sep
test_dir = EXP_LOG_DIR +sep.join(['final_5', sm_name]) +sep

bench = {split: get_bench(f"{EXP_LOG_DIR}/bench/**/{split}.json") for split in ['train', 'val', 'test']}
final = {split: get_final(split, data_names) for split in ['train', 'val', 'test']}

In [16]:
datas = data_names[:2]
year = datas[0][:4]
split = 'test'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
display(year)

for asset_name in ASSETS:
    display(asset_name)
    b = benchyear.loc[asset_name]
    for data_name in datas:
        best = get_best(final[split].xs((asset_name, data_name)), key, sel)
        best = bench_diff(best, b, split)
        display(add_style(best))

'2007'

'sp_500'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,471,5_000159,-0.025000,0.853245,-0.022469,-0.586771,-0.066063,-0.405173,-0.043288,0.504687
cnp,476,4_000507,0.007812,0.673701,-0.037062,0.790722,0.095725,0.796977,0.028813,0.607812
lnp,476,2_000979,-0.029687,0.479624,-0.035903,0.290111,0.034186,0.310888,-0.000010,0.523438
np,471,3_000608,0.015625,0.653798,-0.038259,-0.011856,-0.001596,0.016910,-0.018608,0.807813


,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,376,0_000874,-0.035937,0.275717,-0.028621,0.486679,0.057995,0.470291,0.014970,0.364063
cnp,376,7_000698,-0.004687,0.555841,-0.029360,0.937759,0.114358,0.910871,0.041459,0.670312
lnp,376,5_000371,-0.028125,0.764527,-0.014469,0.274687,0.032333,0.375410,0.009768,0.521875
np,376,5_000371,-0.009375,0.245491,-0.038060,-0.183263,-0.021188,-0.154924,-0.028263,0.946875


'russell_2000'

...

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,452,9_000841,-0.035938,0.432562,-0.049911,0.515291,0.069488,0.531550,0.009646,0.620313
cnp,452,3_000275,-0.056250,0.433504,-0.042715,-0.244278,-0.031795,-0.099949,-0.035951,0.618750
lnp,452,3_000275,-0.031250,0.596130,-0.040621,0.632804,0.085950,0.647192,0.022191,0.593750
np,455,2_000571,-0.046875,0.485509,-0.028005,0.265330,0.035213,0.333195,0.004144,0.696875


,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,420,3_000773,-0.031332,1.088857,-0.016745,1.253292,0.179070,1.246663,0.078294,0.536184
cnp,423,2_000084,-0.019819,1.108334,0.001523,1.180780,0.168076,1.185729,0.082859,0.620066
lnp,423,7_000784,-0.024753,1.309258,-0.030586,0.950542,0.133718,1.018394,0.050069,0.424342
np,423,2_000084,-0.026398,0.610236,-0.032753,0.213399,0.029238,0.321927,-0.000915,0.564145


'dow_jones'

,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,342,5_000515,-0.051562,-0.038238,-0.061240,-0.150160,-0.018347,-0.128949,-0.038696,0.420312
cnp,342,6_000561,0.018750,0.956980,-0.052071,0.416933,0.050669,0.474969,-0.000518,0.959375
lnp,342,1_000675,-0.051562,0.218615,-0.063417,-1.134192,-0.128569,-1.058326,-0.095139,0.707812
np,342,3_000923,0.006250,0.150450,-0.063545,0.053638,0.005978,0.060100,-0.027929,1.000000


,,,test_clf_accuracy,test_kelly_sharpe,test_kelly_cagr,test_binary_sharpe,test_binary_cagr,test_conf_sharpe,test_conf_cagr,test_binary_longfreq
model,study,trial,,,,,,,,
base,445,0_000339,-0.014967,0.032050,-0.045309,0.091467,0.011973,0.089346,-0.016153,0.672697
cnp,447,5_000943,-0.006743,-0.000638,-0.044008,0.373582,0.047370,0.289812,0.001534,0.476974
lnp,447,5_000943,-0.008388,0.658630,-0.027637,0.268346,0.034041,0.381623,0.003829,0.672697
np,445,9_000643,0.004770,0.532195,-0.061643,-0.081369,-0.009199,-0.041126,-0.034214,0.912829


In [25]:
datas = data_names[2:]
year = datas[0][:4]
split = 'test'
benchyear = bench[split].xs(f'{year}_2018_ddir', level=1)
key = f'{split}_kelly_sharpe'
sel = get_sel(split)
display(year)

for asset_name in ASSETS:
    display(asset_name)
    b = benchyear.loc[asset_name]
    for data_name in datas:
        best = get_best(final[split].xs((asset_name, data_name)), key, sel)
        best = bench_diff(best, b, split)
        display(add_style(best))

IndexError: list index out of range